In [ ]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from tqdm import tqdm

In [3]:
data = pd.read_csv('/content/NTO_exp_encoded.csv')
df = pd.DataFrame(data)

In [4]:
numeric_features = ['mass_fraction', 'component_param_value', 'oil_property_value']
categorical_features = ['blend_id', 'component_name', 'component_type_title', 'component_param_title']

In [5]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

In [6]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

processed_data = preprocessor.fit_transform(df)

In [7]:
numeric_columns = numeric_features  # Числовые признаки остаются без изменений
ohe_categories = preprocessor.named_transformers_['cat'].named_steps['onehot'].get_feature_names_out(categorical_features)
all_columns = numeric_columns + list(ohe_categories)
result_df = pd.DataFrame(processed_data, columns=all_columns)

In [8]:
result_df

,mass_fraction,component_param_value,oil_property_value,blend_id_00000d27-5cbc-45d1-8f32-169f158435b3,blend_id_00bcdc21-55e2-4408-bd34-7ece1ef3583a,blend_id_016d3413-ba63-4ee4-8925-b6b9a631e33c,blend_id_021bc9d5-473f-49b2-8bd4-cd40221af7bc,blend_id_02c60023-74c0-49d6-b72f-4df89f1e10de,blend_id_030d200c-7abe-4d1f-a43f-2043bb127ed2,blend_id_038a4242-87e4-4971-a2a4-534da0a39a95,...,component_param_title_d1f1141b-c5ad-483f-956b-576b7e5d395a,component_param_title_d4bc22ae-6679-4162-93df-bc83ab550e5b,component_param_title_d71c61dd-417a-4e43-bafe-e9b9014ac6ce,component_param_title_dab62856-f573-4597-9176-3caf6e3b1b39,component_param_title_e01d985b-e223-424e-8303-e68387f14510,component_param_title_f1acc67e-c976-4aea-83e4-2733b6a499a1,component_param_title_f6a97d54-9e01-4d65-95e6-35f89e576be6,component_param_title_fd593999-0c3f-4709-b586-b4b7fd914fd5,component_param_title_Температура вспышки в закрытом тигле,component_param_title_Температура вспышки в открытом тигле
0,67.00,30.0,4.52,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,33.00,30.0,4.52,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,67.00,30.0,4.52,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,67.00,1612.0,4.52,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,33.00,2360.0,4.52,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101855,49.69,30.0,10.09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101856,19.80,30.0,10.09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101857,29.70,3600.0,10.09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101858,29.70,30.0,10.09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
def remove_outliers_iqr(df, numeric_cols):
    """
    Удаляет выбросы из числовых столбцов с использованием метода IQR.
    """
    df_clean = df.copy()
    for col in numeric_cols:
        Q1 = df_clean[col].quantile(0.25)
        Q3 = df_clean[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.2 * IQR
        upper_bound = Q3 + 1.2 * IQR
        df_clean = df_clean[(df_clean[col] >= lower_bound) & (df_clean[col] <= upper_bound)]
    return df_clean

In [10]:
cleaned_df = remove_outliers_iqr(result_df, numeric_features)

In [11]:
cleaned_df

,mass_fraction,component_param_value,oil_property_value,blend_id_00000d27-5cbc-45d1-8f32-169f158435b3,blend_id_00bcdc21-55e2-4408-bd34-7ece1ef3583a,blend_id_016d3413-ba63-4ee4-8925-b6b9a631e33c,blend_id_021bc9d5-473f-49b2-8bd4-cd40221af7bc,blend_id_02c60023-74c0-49d6-b72f-4df89f1e10de,blend_id_030d200c-7abe-4d1f-a43f-2043bb127ed2,blend_id_038a4242-87e4-4971-a2a4-534da0a39a95,...,component_param_title_d1f1141b-c5ad-483f-956b-576b7e5d395a,component_param_title_d4bc22ae-6679-4162-93df-bc83ab550e5b,component_param_title_d71c61dd-417a-4e43-bafe-e9b9014ac6ce,component_param_title_dab62856-f573-4597-9176-3caf6e3b1b39,component_param_title_e01d985b-e223-424e-8303-e68387f14510,component_param_title_f1acc67e-c976-4aea-83e4-2733b6a499a1,component_param_title_f6a97d54-9e01-4d65-95e6-35f89e576be6,component_param_title_fd593999-0c3f-4709-b586-b4b7fd914fd5,component_param_title_Температура вспышки в закрытом тигле,component_param_title_Температура вспышки в открытом тигле
0,67.00,30.0,4.52,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,33.00,30.0,4.52,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,67.00,30.0,4.52,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,33.00,30.0,4.52,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,67.00,30.0,4.52,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101852,29.70,30.0,10.09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
101855,49.69,30.0,10.09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101856,19.80,30.0,10.09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101858,29.70,30.0,10.09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
X = cleaned_df.drop(columns=['oil_property_value'], axis=0)
y = cleaned_df['oil_property_value']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## CatBoost

In [18]:
from catboost import CatBoostRegressor

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
model = CatBoostClassifier(
    iterations=2500,
    learning_rate=0.01,
    depth=8,
    loss_function='MAE',
    verbose=100,
    early_stopping_rounds=100,
    l2_leaf_reg=3,
    task_type='GPU' if torch.cuda.is_available() else 'CPU'
)

In [ ]:
model.fit(
    X=X_train, y=y_train,
    # eval_set=X_test,
    use_best_model=True,
    plot=True
)

In [ ]:
val_pred = model.predict(y_test)
print(f"CatBoost: {mean_absolute_error(y_test, val_pred):.4f}")

## PyTorch

In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_size = X_train.shape[1]

In [30]:
class DeepRegressor(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, 1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.3),

            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.GELU(),
            nn.Dropout(0.25),

            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.SiLU(),
            nn.Dropout(0.2),

            nn.Linear(256, 1)
        )

    def forward(self, x):
        return self.net(x)

In [31]:
model = DeepRegressor().to(device)
optimizer = optim.AdamW(model.parameters(), lr=0.0005, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5)
criterion = nn.MSELoss()

In [32]:
train_dataset = TensorDataset(
    torch.tensor(X_train.values).float(),  # Преобразование X_train в NumPy массив
    torch.tensor(y_train.values).float()   # Преобразование y_train в NumPy массив
)

val_dataset = TensorDataset(
    torch.tensor(X_test.values).float(),  # Преобразование X_test в NumPy массив
    torch.tensor(y_test.values).float()   # Преобразование y_test в NumPy массив
)

In [33]:
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1024)

In [54]:
best_mse = float('inf')
best_mae = float('inf')

In [55]:
opa_epocha = 150

In [56]:
for epoch in tqdm(range(opa_epocha)):
    model.train()
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(X_batch).squeeze()  # Предсказания модели
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

    # Валидация
    model.eval()
    val_preds, val_targets = [], []
    with torch.no_grad():
        for X_val, y_val_batch in val_loader:
            X_val, y_val_batch = X_val.to(device), y_val_batch.to(device)
            preds = model(X_val).squeeze()  # Предсказания модели
            val_preds.extend(preds.cpu().numpy())
            val_targets.extend(y_val_batch.cpu().numpy())


    mse = mean_squared_error(val_targets, val_preds)
    mae = mean_absolute_error(val_targets, val_preds)
    r2 = r2_score(val_targets, val_preds)
    scheduler.step(mse)  # Оптимизация на основе MSE

    print(f"Epoch {epoch+1}: Val MSE = {mse:.4f}, Val MAE = {mae:.4f}, Val R² = {r2:.4f}")

    if mse < best_mse and mae < best_mae:
        best_mse = mse
        best_mae = mae
        print('New best')
        torch.save(model.state_dict(), 'best_model.pth')

  1%|          | 1/150 [00:01<03:54,  1.58s/it]

Epoch 1: Val MSE = 0.0101, Val MAE = 0.0836, Val R² = 0.9950
New best


  1%|▏         | 2/150 [00:03<04:07,  1.67s/it]

Epoch 2: Val MSE = 0.0094, Val MAE = 0.0764, Val R² = 0.9954
New best


  2%|▏         | 3/150 [00:04<04:02,  1.65s/it]

...

Epoch 148: Val MSE = 0.0081, Val MAE = 0.0720, Val R² = 0.9960


 99%|█████████▉| 149/150 [03:48<00:01,  1.45s/it]

Epoch 149: Val MSE = 0.0196, Val MAE = 0.1119, Val R² = 0.9903


100%|██████████| 150/150 [03:50<00:00,  1.53s/it]

Epoch 150: Val MSE = 0.0096, Val MAE = 0.0812, Val R² = 0.9952


In [58]:
print(f"Validation MSE: {best_mse:.4f}")
print(f"Validation MAE: {best_mae:.4f}")

Validation MSE: 0.0062
Validation MAE: 0.0639


In [59]:
# Создаем экземпляр модели
model = DeepRegressor().to(device)

# Загружаем сохраненные веса
model.load_state_dict(torch.load('/content/best_model.pth'))

# Переводим модель в режим оценки
model.eval()

DeepRegressor(
  (net): Sequential(
    (0): Linear(in_features=649, out_features=1024, bias=True)
    (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.1)
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=1024, out_features=512, bias=True)
    (5): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): GELU(approximate='none')
    (7): Dropout(p=0.25, inplace=False)
    (8): Linear(in_features=512, out_features=256, bias=True)
    (9): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): SiLU()
    (11): Dropout(p=0.2, inplace=False)
    (12): Linear(in_features=256, out_features=1, bias=True)
  )
)

In [ ]:
X_new_test = torch.tensor(X_test.values).float().to(device)

In [66]:
import numpy as np

In [ ]:
with torch.no_grad():  
    predictions = model(X_new_test).squeeze() 

predictions_np = predictions.cpu().numpy()

print(np.array(predictions_np))
print(np.array(y_test))

[6.156739  8.321328  6.643861  ... 8.82716   5.4846935 6.3902664]
[6.25  8.29  6.672 ... 8.83  5.57  6.524]
